# Bucket Optimization
In this task, we will provide a general approach to generating k number of bins or buckets for FICO scores with data-driven ranges that yields the most reliable probability of default for each bin. That is, we want to be more confident in the threshold for default using data and statistics. This minimizes our reliance on domain knowledge, but at the same time increases our vulnerability to model inaccuracy caused by dirty or ineffective datasets.

We will only concern ourselves with generating bin ranges through statistical means. We will use a simpler unsupervised learning approach: 1-dimensional K-means clustering -- to create <i>k</i> number of bins that are defined by proximity with respect to one variable (i.e., FICO score). Each ith bin is assigned a label <i>i</i> (i.e., 1, 2, ... k) such that: $1 \leq k < n $ and is treated an ordinal variable. Lower values represent higher creditworthiness.

### Implementation Plan
1. Sort dataframe by FICO scores
2. Initialize k bin ranges by percentile and set a centroid (mean of each bin range )
3. Create bucket ranges by getting the midpoints between each centroid
4. Set the mean of all samples within each bucket as the new centroid
5. Repeat steps 3 and 4 until the shift between old and updated centroids is negligible (i.e., convergence)

In [1]:
import pandas as pd
import numpy as np

# load data
file_path = "data/Loan_Data.csv"
df = pd.read_csv(file_path)

In [2]:
def initialize_centroids(scores, k):
    percentiles = np.linspace(0, 100, k + 2)[1:-1]
    return np.percentile(scores, percentiles)


def assign_buckets(scores, centroids):
    boundaries = [(centroids[i] + centroids[i+1]) / 2 for i in range(len(centroids)-1)]
    reversed_boundaries = boundaries[::-1]
    return np.digitize(scores, reversed_boundaries)


def update_centroids(scores, labels, k):
    new_centroids = []
    
    for j in range(k):
        cluster_points = scores[labels == j]
        
        if len(cluster_points) > 0:
            new_centroids.append(cluster_points.mean())
        
        else: # fallback if bucket is empty
            new_centroids.append(np.random.choice(scores))
    return np.array(new_centroids)


def kmeans_1d(df, col, k, max_iter=100, tol=1e-3):
    """
    Perform 1D k-means clustering on a DataFrame column.
    Returns the DataFrame with a new 'rating' column.
    """
    scores = df[col].values
    centroids = initialize_centroids(scores, k)

    for _ in range(max_iter):
        labels = assign_buckets(scores, centroids)
        new_centroids = update_centroids(scores, labels, k)
        shift = np.abs(new_centroids - centroids).max()
        
        # if updated centroid displacement is small enough, end
        if shift < tol:
            break
        centroids = new_centroids

    # Assign final buckets as ratings (1 = worst, k = best)
    labels = assign_buckets(scores, centroids)
    df = df.copy()
    df['rating'] = labels + 1
    return df, centroids


In [3]:
df_buckets, centroids = kmeans_1d(df, col="fico_score", k=3)
sorted_df_buckets = df_buckets.sort_values("fico_score")
print("Centroids:", centroids)
sorted_df_buckets

Centroids: [561.62142585 636.59986787 709.32440056]


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default,rating
2092,7264776,1,4457.914800,12233.495010,98913.32028,3,408,0,3
6556,6901345,3,5281.352243,16411.518010,79905.09892,1,409,1,3
7001,2585781,4,6734.984475,26384.584390,97668.03091,2,418,1,3
5521,1252008,5,5176.915602,22990.265430,82417.59227,2,425,1,3
2629,1337395,5,4271.314690,22756.281030,83475.30929,4,438,1,3
...,...,...,...,...,...,...,...,...,...
703,2967189,2,4803.995367,12920.809080,82500.95110,8,828,0,1
7575,2214311,1,3590.741275,8071.647089,71796.25458,8,831,0,1
4768,2713140,0,3566.709054,4400.583413,60204.65020,4,831,0,1
9660,5062008,0,780.213472,273.300211,10258.24788,3,835,0,1


From the preview, our cluster approach is working at the extreme values, in which lower FICO scores are predicted to default and vice versa. However, this approach leads to being stuck at local minimia. To address this, we can employ dynamic programming that minimizes Mean Squared Error (MSE).

In [4]:
def compute_sse(prefix_sum, prefix_sq, a, b):
    """
    Compute SSE (sum of squared errors) for scores[a:b].
    prefix_sum, prefix_sq are cumulative sums over sorted scores.
    """
    n = b - a
    total = prefix_sum[b] - prefix_sum[a]
    total_sq = prefix_sq[b] - prefix_sq[a]
    mean = total / n
    
    return total_sq - 2 * mean * total + n * mean**2


def optimal_bucketing(scores, k):
    """
    Dynamic programming solution for 1D k-means (MSE minimization).
        Input: sorted numpy array 'scores', number of buckets k
        Output: bucket boundaries and assignments
    """
    n = len(scores)

    # Prefix sums for fast SSE computation
    prefix_sum = np.zeros(n + 1)
    prefix_sq = np.zeros(n + 1)
    for i in range(n):
        prefix_sum[i+1] = prefix_sum[i] + scores[i]
        prefix_sq[i+1] = prefix_sq[i] + scores[i]**2

    # DP table and backtracking
    dp = np.full((n+1, k+1), np.inf)
    backtrack = np.zeros((n+1, k+1), dtype=int)
    dp[0,0] = 0

    for j in range(1, k+1):
        for i in range(j, n+1):
            for t in range(j-1, i):
                cost = dp[t,j-1] + compute_sse(prefix_sum, prefix_sq, t, i)
                if cost < dp[i,j]:
                    dp[i,j] = cost
                    backtrack[i,j] = t
    # Recover boundaries
    boundaries = []
    i, j = n, k
    while j > 0:
        t = backtrack[i,j]
        boundaries.append(scores[t])
        i, j = t, j-1
    boundaries = sorted(boundaries[1:] + [scores[-1]])  # drop first, keep max

    return boundaries


def assign_buckets(df, col, boundaries):
    """
    Assign each row in df to a bucket based on boundaries.
    Lower buckets = lower FICO = worse rating.
    """
    df = df.copy()
    df['rating'] = np.digitize(df[col], boundaries, right=True) + 1
    return df


# === Example Usage ===
if __name__ == "__main__":
    k = 3
    sorted_scores = np.sort(df['fico_score'].values)
    boundaries = optimal_bucketing(sorted_scores, k)

    df_binned = assign_buckets(df, "fico_score", boundaries)

    print("Bucket boundaries:", boundaries)
    print(df_binned.sort_values("fico_score"))


Bucket boundaries: [408, 600, 850]
      customer_id  credit_lines_outstanding  loan_amt_outstanding  \
2092      7264776                         1           4457.914800   
6556      6901345                         3           5281.352243   
7001      2585781                         4           6734.984475   
5521      1252008                         5           5176.915602   
2629      1337395                         5           4271.314690   
...           ...                       ...                   ...   
703       2967189                         2           4803.995367   
7575      2214311                         1           3590.741275   
4768      2713140                         0           3566.709054   
9660      5062008                         0            780.213472   
2659      1096584                         0           4107.822428   

      total_debt_outstanding       income  years_employed  fico_score  \
2092            12233.495010  98913.32028               3      